In [1]:
import os
import random
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

ROOT_DIR = "./data"
RANDOM_STATE = 110

In [14]:
df = pd.read_csv(os.path.join(ROOT_DIR, "Fill1 dispensing.csv"), low_memory=False)
df.head()

,Wip Line,Process Desc.,Equipment,Model.Suffix,Workorder,LOT ID,Set ID,Box ID,Collect Date,Insp. Seq No.,...,Collect Result.27,Unit Time.27,Judge Value.27,Collect Result.28,Unit Time.28,Judge Value.28,Collect Result.29,Unit Time.29,Judge Value.29,Unnamed: 101
0,IVI-OB6,Fill1 Dispenser,Fill1 dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000002,OP753345013050000002,NaN,2023-05-04 08:59:27,1,...,1,NaN,NaN,1,NaN,NaN,7,NaN,NaN,NaN
1,IVI-OB6,Fill1 Dispenser,Fill1 dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000003,OP753345013050000003,NaN,2023-05-04 09:13:39,1,...,2,NaN,NaN,1,NaN,NaN,7,NaN,NaN,NaN
2,IVI-OB6,Fill1 Dispenser,Fill1 dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000004,OP753345013050000004,NaN,2023-05-04 09:15:42,1,...,3,NaN,NaN,1,NaN,NaN,7,NaN,NaN,NaN
3,IVI-OB6,Fill1 Dispenser,Fill1 dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000005,OP753345013050000005,NaN,2023-05-04 09:17:46,1,...,4,NaN,NaN,1,NaN,NaN,7,NaN,NaN,NaN
4,IVI-OB6,Fill1 Dispenser,Fill1 dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000006,OP753345013050000006,NaN,2023-05-04 09:19:50,1,...,5,NaN,NaN,1,NaN,NaN,7,NaN,NaN,NaN


In [16]:
df = pd.read_excel(os.path.join(ROOT_DIR, "Fill1 dispensing.xlsx"), header=[0,1])

In [17]:
df.head()

Collect Item:                                                    \
           Wip Line    Process Desc.           Equipment Model.Suffix   
0           IVI-OB6  Fill1 Dispenser  Fill1 dispenser #1  AJX75334501   
1           IVI-OB6  Fill1 Dispenser  Fill1 dispenser #1  AJX75334501   
2           IVI-OB6  Fill1 Dispenser  Fill1 dispenser #1  AJX75334501   
3           IVI-OB6  Fill1 Dispenser  Fill1 dispenser #1  AJX75334501   
4           IVI-OB6  Fill1 Dispenser  Fill1 dispenser #1  AJX75334501   
...             ...              ...                 ...          ...   
61923       IVI-OB6  Fill1 Dispenser  Fill1 dispenser #2  AJX75334505   
61924       IVI-OB6  Fill1 Dispenser  Fill1 dispenser #1  AJX75334505   
61925       IVI-OB6  Fill1 Dispenser  Fill1 dispenser #2  AJX75334505   
61926       IVI-OB6  Fill1 Dispenser  Fill1 dispenser #1  AJX75334505   
61927       IVI-OB6  Fill1 Dispenser  Fill1 dispenser #2  AJX75334505   

                                                                      \
        Workorder                LOT ID                Set ID Box ID   
0      3F1X5847-2  OP753345013050000002  OP753345013050000002    NaN   
1      3F1X5847-2  OP753345013050000003  OP753345013050000003    NaN   
2      3F1X5847-2  OP753345013050000004  OP753345013050000004    NaN   
3      3F1X5847-2  OP753345013050000005  OP753345013050000005    NaN   
4      3F1X5847-2  OP753345013050000006  OP753345013050000006    NaN   
...           ...                   ...                   ...    ...   
61923  4F1XB738-1  OP753345054040002685  OP753345054040002685    NaN   
61924  4F1XB738-1  OP753345054040002686  OP753345054040002686    NaN   
61925  4F1XB738-1  OP753345054040002687  OP753345054040002687    NaN   
61926  4F1XB738-1  OP753345054040002688  OP753345054040002688    NaN   
61927  4F1XB738-1  OP753345054040002689  OP753345054040002689    NaN   

                                          ... DISCHARGED SPEED OF RESIN  \
              Collect Date Insp. Seq No.  ...         Collect Result.27   
0      2023-05-04 08:59:27             1  ...                         1   
1      2023-05-04 09:13:39             1  ...                         2   
2      2023-05-04 09:15:42             1  ...                         3   
3      2023-05-04 09:17:46             1  ...                         4   
4      2023-05-04 09:19:50             1  ...                         5   
...                    ...           ...  ...                       ...   
61923         45410.772546             1  ...                        11   
61924         45410.773287             1  ...                         5   
61925         45410.774028             1  ...                        12   
61926         45410.774688             1  ...                         6   
61927         45410.775359             1  ...                        13   

                                                                  \
      Unit Time.27 Judge Value.27 Collect Result.28 Unit Time.28   
0              NaN            NaN                 1          NaN   
1              NaN            NaN                 1          NaN   
2              NaN            NaN                 1          NaN   
3              NaN            NaN                 1          NaN   
4              NaN            NaN                 1          NaN   
...            ...            ...               ...          ...   
61923          NaN            NaN               435          NaN   
61924          NaN            NaN               436          NaN   
61925          NaN            NaN               437          NaN   
61926          NaN            NaN               438          NaN   
61927          NaN            NaN               439          NaN   

                                        WorkMode                            
      Judge Value.28 Collect Result.29 Unit Time Judge Value Judge Value.1  
0                NaN                 7       NaN         NaN           NaN  
1                NaN       

In [3]:
nan_columns = df.columns[df.isnull().any()]
# 전부 NaN가 아니더라도 최소 19126개 NaN 있어서 전부 삭제

In [4]:
df.drop(nan_columns, axis=1, inplace=True)
df.drop(["Process Desc.", "LOT ID"], axis=1, inplace=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61928 entries, 0 to 61927
Columns: 102 entries, ('Collect Item:', 'Wip Line') to ('WorkMode', 'Judge Value.1')
dtypes: float64(83), int64(8), object(11)
memory usage: 48.2+ MB


In [6]:
df.describe()

,Insp. Seq No.,Collect Result,Collect Result.1,Collect Result.2,Collect Result.3,Collect Result.4,Collect Result.5,Collect Result.6,Collect Result.8,Collect Result.9,...,Collect Result.20,Collect Result.21,Collect Result.22,Collect Result.23,Collect Result.24,Collect Result.25,Collect Result.26,Collect Result.27,Collect Result.28,Collect Result.29
count,61928.000000,61928.000000,61928.000000,61928.000000,61928.000000,61928.000000,61928.000000,61928.000000,61928.000000,61928.000000,...,61928.000000,61928.000000,61928.000000,61928.000000,61928.000000,61928.000000,61928.000000,61928.000000,61928.000000,61928.000000
mean,1.054337,10.611552,13.342388,3.889714,13.371507,12.640055,3.684993,12.667586,676.973652,333.495803,...,92.777949,67.605594,174.303811,189.290596,76.929660,109.454292,36.972610,76.432018,107.602183,3.682648
std,0.244769,0.259017,0.666852,0.691247,0.521997,0.534396,0.650703,0.363049,189.145437,150.143721,...,36.193503,20.808799,97.041982,117.850781,37.262596,31.345702,26.657068,107.202876,131.452729,3.018778
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,156.100000,157.000000,...,50.000000,50.000000,92.200000,50.000000,50.000000,40.300000,0.000000,0.000000,0.000000,1.000000
25%,1.000000,10.600000,12.900000,3.600000,12.900000,12.250000,3.420000,12.250000,458.800000,157.000000,...,50.000000,50.000000,92.200000,50.000000,50.000000,90.100000,7.000000,7.000000,1.000000,1.000000
50%,1.000000,10.600000,13.400000,3.600000,13.500000,12.730000,3.420000,12.730000,837.700000,458.100000,...,123.400000,50.000000,92.200000,289.000000,50.000000,128.000000,54.100000,14.000000,44.000000,1.000000
75%,1.000000,10.600000,13.500000,3.900000,13.500000,12.820000,3.700000,12.820000,838.400000,458.400000,...,123.400000,92.200000,289.000000,289.000000,85.000000,128.000000,57.500000,129.000000,194.000000,7.000000
max,4.000000,10.900000,17.400000,14.800000,17.400000,14.060000,14.060000,14.060000,839.000000,838.400000,...,123.400000,92.200000,289.000000,289.000000,145.000000,175.800000,214.900000,489.000000,608.000000,17.000000


In [7]:
df.isna().sum()

Wip Line             0
Equipment            0
Model.Suffix         0
Workorder            0
Set ID               0
Collect Date         0
Insp. Seq No.        0
Insp Judge Code      0
Collect Result       0
Collect Result.1     0
Collect Result.2     0
Collect Result.3     0
Collect Result.4     0
Collect Result.5     0
Collect Result.6     0
Collect Result.8     0
Collect Result.9     0
Collect Result.10    0
Collect Result.11    0
Collect Result.12    0
Collect Result.13    0
Collect Result.14    0
Collect Result.15    0
Collect Result.16    0
Collect Result.17    0
Collect Result.18    0
Collect Result.19    0
Collect Result.20    0
Collect Result.21    0
Collect Result.22    0
Collect Result.23    0
Collect Result.24    0
Collect Result.25    0
Collect Result.26    0
Collect Result.27    0
Collect Result.28    0
Collect Result.29    0
dtype: int64

In [10]:
df["Equipment"].unique()

array(['Fill1 dispenser #1', 'Fill1 dispenser #2'], dtype=object)

In [11]:
df["Equipment"] = df["Equipment"].str.split("#", expand=True)[1]

In [8]:
df["Model.Suffix - 1"] = df["Model.Suffix"].str[:4]
# e.g. AJX7
df["Workorder - 1"] = df["Workorder"].str.split('-', expand=True)[0].str[:4]
# e.g. 3FIX

In [9]:
df["Collect Date"] = pd.to_datetime(df["Collect Date"], format="%Y-%m-%d %H:%M:%S")

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61928 entries, 0 to 61927
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Wip Line           61928 non-null  object        
 1   Equipment          61928 non-null  object        
 2   Model.Suffix       61928 non-null  object        
 3   Workorder          61928 non-null  object        
 4   Set ID             61928 non-null  object        
 5   Collect Date       61928 non-null  datetime64[ns]
 6   Insp. Seq No.      61928 non-null  int64         
 7   Insp Judge Code    61928 non-null  object        
 8   Collect Result     61928 non-null  float64       
 9   Collect Result.1   61928 non-null  float64       
 10  Collect Result.2   61928 non-null  float64       
 11  Collect Result.3   61928 non-null  float64       
 12  Collect Result.4   61928 non-null  float64       
 13  Collect Result.5   61928 non-null  float64       
 14  Collec